In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
import torchvision.utils as vutils

# ハイパーパラメータ設定
batch_size = 128
image_size = 28   # MNISTの場合(28x28)
nz = 100          # 潜在ベクトル (ノイズ) の次元数
num_epochs = 5
lr = 0.0002
beta1 = 0.5

# GPUが使える場合はGPUを使用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1) データセットの用意 (例: MNIST)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # ピクセルを -1～1 に正規化
])

dataset = MNIST(root="mnist_data", download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 2) Generatorの定義 (DCGAN風の例)
class Generator(nn.Module):
    def __init__(self, nz=100):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # 入力: (nz) -> 出力: 64チャンネル x 7 x 7 (今回はMNISTを例にしているため、少し簡易的)
            nn.ConvTranspose2d(nz, 64, 7, 1, 0, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            # 64 x 7 x 7 -> 32 x 14 x 14
            nn.ConvTranspose2d(64, 32, 4, 2, 1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            # 32 x 14 x 14 -> 1 x 28 x 28
            nn.ConvTranspose2d(32, 1, 4, 2, 1, bias=False),
            nn.Tanh()
        )
        
    def forward(self, x):
        return self.main(x)

# 3) Discriminatorの定義
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # 入力: 1 x 28 x 28 -> 32 x 14 x 14
            nn.Conv2d(1, 32, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # 32 x 14 x 14 -> 64 x 7 x 7
            nn.Conv2d(32, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            # 64 x 7 x 7 -> 1 x 1 x 1 (最終出力はスカラー: 本物/偽物の判定)
            nn.Conv2d(64, 1, 7, 1, 0, bias=False),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.main(x).view(-1)  # shapeを[batch_size]に

# モデル初期化
netG = Generator(nz).to(device)
netD = Discriminator().to(device)

# 損失関数とオプティマイザ
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

# 学習のメインループ
for epoch in range(num_epochs):
    for i, (real_images, _) in enumerate(dataloader):
        # ---------------------------------------------
        # (1) Discriminatorの学習
        # ---------------------------------------------
        netD.zero_grad()
        
        # 本物画像での訓練
        real_images = real_images.to(device)
        batch_size_cur = real_images.size(0)
        labels_real = torch.ones(batch_size_cur, device=device)  # 本物ラベル=1
        output = netD(real_images)
        lossD_real = criterion(output, labels_real)
        lossD_real.backward()

        # 偽画像での訓練
        noise = torch.randn(batch_size_cur, nz, 1, 1, device=device)
        fake_images = netG(noise)
        labels_fake = torch.zeros(batch_size_cur, device=device) # 偽ラベル=0
        output = netD(fake_images.detach())  # detach()でGeneratorへの伝搬をストップ
        lossD_fake = criterion(output, labels_fake)
        lossD_fake.backward()
        
        # Discriminatorを更新
        optimizerD.step()

        # ---------------------------------------------
        # (2) Generatorの学習
        # ---------------------------------------------
        netG.zero_grad()
        # Generatorは「生成画像をDiscriminatorが本物(=1)と判断する」ようにしたい
        labels_gen = torch.ones(batch_size_cur, device=device)
        output = netD(fake_images)  # 偽画像を再度Discriminatorに通す
        lossG = criterion(output, labels_gen)
        lossG.backward()
        
        # Generatorを更新
        optimizerG.step()

        # ---------------------------------------------
        # ログ＆可視化用
        # ---------------------------------------------
        if i % 200 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}] Step [{i}/{len(dataloader)}] "
                  f"Loss_D: {(lossD_real+lossD_fake).item():.4f} Loss_G: {lossG.item():.4f}")

    # エポックの最後にサンプル生成
    with torch.no_grad():
        fixed_noise = torch.randn(64, nz, 1, 1, device=device)
        fake = netG(fixed_noise).cpu()
    # 生成画像をグリッドで保存（または表示）
    vutils.save_image(fake, f"epoch_{epoch+1}.png", normalize=True)
